## Classiq Implementation For The Research Paper: Symmetry enhanced variational quantum spin eigensolver

In [53]:
# Handle imports
from classiq import * # Using Classiq Version 0.52.0 Release
import numpy as np
from typing import cast
# import scipy as sp

In [54]:
# Circuit design variables
n_bits = 4
n_layers = 5
J = 1

# Dummy parameters for testing implementation of ciruit
theta_1 = np.pi
theta_2 = np.pi
theta_3 = np.pi
theta_4 = np.pi
theta_5 = np.pi
theta_6 = np.pi
theta_7 = np.pi

training_params = [theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7]

# Initialization circuit layer
@qfunc
def Init(q: QArray) -> None:
    # First, do PauliX on all qubits
    for i in range(n_bits):
        X(q[i])

    # Next, add the Hadamards and CNOT
    for i in range(0, n_bits, 2):
        hadamard_transform(q[i])
        CX(q[i],q[i + 1])


# N block function
@qfunc
def N_block(q0: QBit, q1: QBit, param: CReal) -> None:
    RZ(np.pi / 2, q1)

    CX(q1, q0)
     
    RZ(2 * param - np.pi/2, q0)
    RY(np.pi / 2 - 2 * param, q1)
    
    CX(q0, q1)

    RY(2 * param - np.pi/2, q1)

    CX(q1, q0)
    RZ(-np.pi / 2, q0)


# Main ansatz layer
@qfunc
def Sz_conserving_layer(q: QArray, params: CArray[CReal]) -> None:
    # Apply the N_block layer for entangling
    N_block(q[0],q[1], params[0])
    N_block(q[2],q[3], params[1])
    N_block(q[1],q[2], params[2])

    # Apply the Phase Gates
    for i in range(n_bits):
        PHASE(params[3+i], q[i])

# Function to build the Paulis string
def build_pauli_string(N: int, J: float) -> list[tuple[str, float]]:
    pauli_list = []
    
    # Loop over each neighboring pair of spins (i, i+1)
    for i in range(N-1):
        # Create Pauli strings for sigma_x, sigma_y, sigma_z interactions
        for pauli in ['X', 'Y', 'Z']:
            # Identity for other positions, Pauli for position i and i+1
            identity = ['I'] * N
            identity[i] = pauli
            identity[i+1] = pauli
            
            # Join Pauli string and add it to the list with coefficient J
            pauli_string = ''.join(identity)
            pauli_list.append((pauli_string, J))
    
    return pauli_list


# Definition of Pauli Terms
CHAR_TO_STUCT_DICT = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

# Functions for creating the Hamiltonian from the Paulis String
def pauli_str_to_enums(pauli):
    return [CHAR_TO_STUCT_DICT[s] for s in pauli]

def pauli_list_to_hamiltonian(pauli_list):
    return [
        PauliTerm(
            pauli=pauli_str_to_enums(pauli), coefficient=cast(complex, coeff).real
        )
        for pauli, coeff in pauli_list
    ]

# Main Program
@qfunc
def main():
    # Create the array of bits of proper size
    q = QArray("q")
    allocate(n_bits, q)
    
    # Create the pauli list
    pauli_list = build_pauli_string(n_bits,J)
    # Create the Hamiltonian
    heis_ham = pauli_list_to_hamiltonian(pauli_list)

    print(heis_ham)
    # Do the initialization circuit
    Init(q)

    # Do n layers of the Sz_conserving Ansatz
    for _ in range(n_layers):
        Sz_conserving_layer(q,training_params)


In [55]:
# Simulation
mod = create_model(main)
qprog = synthesize(mod)
show(qprog)

[PauliTerm(pauli=[<Pauli.X: 1>, <Pauli.X: 1>, <Pauli.I: 0>, <Pauli.I: 0>], coefficient=1), PauliTerm(pauli=[<Pauli.Y: 2>, <Pauli.Y: 2>, <Pauli.I: 0>, <Pauli.I: 0>], coefficient=1), PauliTerm(pauli=[<Pauli.Z: 3>, <Pauli.Z: 3>, <Pauli.I: 0>, <Pauli.I: 0>], coefficient=1), PauliTerm(pauli=[<Pauli.I: 0>, <Pauli.X: 1>, <Pauli.X: 1>, <Pauli.I: 0>], coefficient=1), PauliTerm(pauli=[<Pauli.I: 0>, <Pauli.Y: 2>, <Pauli.Y: 2>, <Pauli.I: 0>], coefficient=1), PauliTerm(pauli=[<Pauli.I: 0>, <Pauli.Z: 3>, <Pauli.Z: 3>, <Pauli.I: 0>], coefficient=1), PauliTerm(pauli=[<Pauli.I: 0>, <Pauli.I: 0>, <Pauli.X: 1>, <Pauli.X: 1>], coefficient=1), PauliTerm(pauli=[<Pauli.I: 0>, <Pauli.I: 0>, <Pauli.Y: 2>, <Pauli.Y: 2>], coefficient=1), PauliTerm(pauli=[<Pauli.I: 0>, <Pauli.I: 0>, <Pauli.Z: 3>, <Pauli.Z: 3>], coefficient=1)]


/usr/bin/xdg-open: 882: x-www-browser: Permission denied
/usr/bin/xdg-open: 882: firefox: Permission denied
/usr/bin/xdg-open: 882: iceweasel: Permission denied
/usr/bin/xdg-open: 882: seamonkey: Permission denied
/usr/bin/xdg-open: 882: mozilla: Permission denied
/usr/bin/xdg-open: 882: epiphany: Permission denied
/usr/bin/xdg-open: 882: konqueror: Permission denied
/usr/bin/xdg-open: 882: chromium: Permission denied
/usr/bin/xdg-open: 882: chromium-browser: Permission denied
/usr/bin/xdg-open: 882: google-chrome: Permission denied
/usr/bin/xdg-open: 882: www-browser: Permission denied
/usr/bin/xdg-open: 882: links2: Permission denied
/usr/bin/xdg-open: 882: elinks: Permission denied
/usr/bin/xdg-open: 882: links: Permission denied
/usr/bin/xdg-open: 882: lynx: Permission denied
/usr/bin/xdg-open: 882: w3m: Permission denied
xdg-open: no method available for opening 'https://platform.classiq.io/circuit/96a2e998-0a32-449a-8a0c-1751176637d1?version=0.53.0'
